In [53]:
# Import stanza
import stanza

# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

# Import client module
from stanza.server import CoreNLPClient

# Import tqdm for loading 
from tqdm import tqdm

# Import pandas for Dataframe
import pandas as pd

# Setting AllenNLP
from allennlp.predictors.predictor import Predictor
from allennlp_models.coref.predictors.coref import CorefPredictor
from allennlp_models.coref.dataset_readers.conll import ConllCorefReader
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")

# Import Text
text = "The 1Malaysia Development Berhad scandal (1MDB scandal) is an ongoing political scandal occurring in Malaysia. In 2015, Malaysia's then Prime Minister Najib Razak  was accused of channelling over RM 2.67 billion (approximately US $700 million) from 1Malaysia Development Berhad (1MDB), a government run strategic development company (masterminded by Low Taek Jho, commonly referred to as Jho Low), to his personal bank accounts. The event triggered widespread criticism among Malaysians, with many calling for Najib Razak's resignation including Mahathir Mohamad, one of Najib Razak's predecessors as Prime Minister, who later defeated Najib Razak in the 2018 general election and returned to power. Anwar Ibrahim, a political leader in opposition to Najib Razak, openly questioned 1MDB's credentials. He told Parliament that, according to records held by the Companies Commission, the company has no business address and no appointed auditor. According to its publicly filed accounts, 1MDB had nearly RM 42 billion (US $11.73 billion) in debt. Some of this debt resulted from a $3 billion state guaranteed 2013 bond issue led by the investment bank Goldman Sachs, who have been reported as receiving fees of up to $300 million for the deal, although the bank disputes this figure. The Malaysian Conference of Rulers called for prompt investigation of the scandal, saying that it was causing a crisis of confidence in Malaysia. After the 2018 election, the newly elected Prime Minister, Mahathir Mohamad, reopened the investigation into the 1MDB scandal. Malaysian authorities barred Najib Razak from leaving the country, then seized cash and valuable items from premises linked to him. Najib Razak was charged with criminal breach of trust, money laundering and abuse of power, while Jho Low was charged with money laundering. The US Department of Justice pursued its own investigation into 1MDB, alleging that more than US $4.5 billion was diverted from 1MDB by Jho Low and other conspirators including officials from Malaysia, Saudi Arabia and the United Arab Emirates. Najib Razak was subsequently found guilty of seven charges connected to SRC International, a dummy corporation associated with 1MDB, and was sentenced to twelve years imprisonment."

# View Coref Clusters and their positions
tks = predictor.predict(text)['document']
cluster = predictor.predict(text)['clusters']
coref_positions = []
corrected_coref_positions = []
start = 0
import re

print('Extracting Coreference Clusters and Positions')
for i in tqdm(range(len(cluster))):
    coref_position = []
    for j in range(len(cluster[i])):
        coref = ' '.join(tks[cluster[i][j][0]:cluster[i][j][1] + 1])
        #coref = re.sub("\s*(\W)\s*",r"\1",coref)
        coref = re.sub(" , ", r", ", coref)
        coref = re.sub(" ,", r", ", coref)
        coref = re.sub(" 's", r"'s", coref)
        coref = re.sub("[(] ", r"(", coref)
        coref = re.sub(" [)]", r")", coref)
        coref = re.sub("[$] ", r"$", coref)
        coref = re.sub("RM   ", r"RM ", coref)
        
        start_index = text.find(coref, start)
        if start_index == -1:
            start_index = text.find(coref)
        end_index   = start_index+len(coref)
        start = end_index + 1
        position = list([coref, start_index, end_index])
        coref_position.append(position)
    coref_positions.append(coref_position)
print('\nDone Extraction!')

pd.DataFrame(coref_positions)

#resolved = predictor.coref_resolved(text)  

100%|██████████| 16/16 [00:00<00:00, 6637.21it/s]

Extracting Coreference Clusters and Positions

Done Extraction!


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"[1Malaysia Development Berhad, 4, 32]","[1MDB, 42, 46]","[1Malaysia Development Berhad (1MDB), a govern...","[1MDB's, 786, 792]","[the company, 886, 897]","[its, 963, 966]","[1MDB, 992, 996]","[1MDB, 1549, 1553]","[1MDB, 1900, 1904]","[1MDB, 1964, 1968]","[1MDB, 2208, 2212]","[1MDB, 2528, 2532]","[1MDB, 2637, 2641]","[1MDB, 2704, 2708]","[1MDB, 2936, 2940]","[its, 3011, 3014]"
1,"[Malaysia, 5, 13]","[Malaysia's, 120, 130]","[Malaysia, 250, 258]","[the country, 1617, 1628]","[Malaysia, 2028, 2036]",None,None,None,None,None,None,None,None,None,None,None
2,"[Malaysia's then Prime Minister Najib Razak, 1...","[his, 401, 404]","[Najib Razak's, 510, 523]","[Najib Razak's, 574, 587]","[Najib Razak, 639, 650]","[Najib Razak, 755, 766]","[Najib Razak, 1592, 1603]","[him, 1690, 1693]","[Najib Razak, 1695, 1706]","[Najib Razak, 2081, 2092]","[Najib Razak's, 2441, 2454]","[Najib Razak, 2809, 2820]",None,None,None,None
3,"[channelling, 179, 190]","[The event, 429, 438]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"[Anwar Ibrahim, a political leader in oppositi...","[He, 806, 808]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,[nearly RM 42 billion (US $11.73 billion) in d...,"[this debt, 1059, 1068]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,"[the investment bank Goldman Sachs, 1136, 1169]","[the bank, 1257, 1265]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,"[up to $300 million, 1215, 1233]","[this figure, 1275, 1286]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,[The 1Malaysia Development Berhad scandal (1MD...,"[the scandal, 1358, 1369]","[it, 1383, 1385]","[the 1MDB scandal, 1545, 1561]",None,None,None,None,None,None,None,None,None,None,None,None
9,"[the 2018 general election, 654, 679]","[the 2018 election, 1441, 1458]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
